In [1]:
import os

list_file = sorted(list(os.walk('./file/2.url'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx', 'crawler_2.xlsx']

In [2]:
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/2.url/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Url'].tolist()

总数量：12


['https://www.auveco.com/metric-fastener-assortment-w-o-bin-1043',
 'https://www.auveco.com/metric-fastener-assortment-in-two-40-compartment-bins-1042',
 'https://www.auveco.com/metric-fastener-asst-reg-pitch-w-o-bin-1041',
 'https://www.auveco.com/metric-fast-asst-in-40-comp-bin-1040',
 'https://www.auveco.com/uss-fastener-asst-wo-bin-1031',
 'https://www.auveco.com/gr-8-fastener-asst-in-40-compartment-bin-1030',
 'https://www.auveco.com/gr-5-uss-fast-asst-w-o-bin-1021',
 'https://www.auveco.com/gr-5-fast-asst-in-40-comp-bin-1020',
 'https://www.auveco.com/import-retainer-assortment-w-o-bin-1055',
 'https://www.auveco.com/import-retainer-assortment-in-36-comp-bin-1054',
 'https://www.auveco.com/domestic-retainer-assortment-w-o-bin-1053',
 'https://www.auveco.com/domestic-retainer-assortment-in-36-compartment-bin-1052']

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

b = 0
while True:
    b += 1
    print(b)
    try:
        resp = requests.get(input_.loc[a, 'Url'],
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(10, 10))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            with open('./part.html', 'w', encoding='UTF-8') as file:
                file.write(str(soup))

            if html.xpath('//h3/text()')[0].strip() == 'Can\'t find the item you are looking for on this list? Download this':
                break

            title = html.xpath('//h1[@itemprop="name"]/text()')[0].strip()
            if html.xpath('//li[@class="global-views-breadcrumb-item"]/a[@data-touchpoint="home"]/text()')[0].strip() == 'Home' and html.xpath('//li[@class="global-views-breadcrumb-item-active"]/text()')[0].strip() == title:
                break
    except KeyboardInterrupt:
        break
    except:
        continue

resp

1
2
3
4
5
6
7
8
9
10
11
12
13


<Response [200]>

In [ ]:
html.xpath('//h3/text()')[0].strip() == 'Can\'t find the item you are looking for on this list? Download this'

In [8]:
list_src = html.xpath('//ul[@class="bxslider"]/li[@class="product-details-image-gallery-container"]/noscript/img[@itemprop="image"]/@src') + html.xpath('//div[@class="product-details-image-gallery"]/div[@class="product-details-image-gallery-detailed-image"]/noscript/img[@class="center-block"]/@src')
list_src = sorted(list(set(list_src)), key=list_src.index)

list_src

['https://www.auveco.com/site/item-images/1043_image-1.jpg?resizeid=3&resizeh=600&resizew=600',
 'https://www.auveco.com/site/item-images/1043_image-2.jpg?resizeid=3&resizeh=600&resizew=600']

In [9]:
dict_src = {}
for i, src in zip(range(len(list_src)), list_src):
    dict_src[str(i)] = src.strip()

dict_src

{'0': 'https://www.auveco.com/site/item-images/1043_image-1.jpg?resizeid=3&resizeh=600&resizew=600',
 '1': 'https://www.auveco.com/site/item-images/1043_image-2.jpg?resizeid=3&resizeh=600&resizew=600'}

In [10]:
sku = html.xpath('//span[@itemprop="sku"]/text()')[0].strip()

sku

'1043'

In [11]:
dict_description, list_tr = {}, html.xpath('//form[@id="product-details-full-form"]/table[@class="specs-table table table-striped"]/tbody/tr')
for i, tr in zip(range(len(list_tr)), list_tr):
    dict_description[str(i)] = {tr.xpath('./td[1]/text()')[0].strip(): tr.xpath('./td[2]/text()')[0].strip()}

dict_description

{'0': {'Pcs/Unit': '1'},
 '1': {'Includes': '1995 Pieces, Planogram And Reorder Labels, Without The 72B-9 Bin'},
 '2': {'Country': 'Mexico'}}

In [12]:
list_oem = [text.strip() for text in html.xpath('//form[@id="product-details-full-form"]/div[@data-view="Additional.Detail.Fields"]/div/table[@class="specs-table table table-striped"]/tbody/tr/td[1]/text()')]
list_oem = sorted(list(set(list_oem)), key=list_oem.index)

oem = ';'.join(list_oem)

oem

''

In [13]:
import json

df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                         'SKU': sku,
                         'Title': title,
                         'OEM': oem,
                         'Picture': '',
                         'Url': input_.loc[a, 'Url'],
                         'Json_Src': json.dumps(dict_src),
                         'Json_Description': json.dumps(dict_description)}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,SKU,Title,OEM,Picture,Url,Json_Src,Json_Description
0,1,1043,Metric Fastener Assortment withoutut Compartme...,,,https://www.auveco.com/metric-fastener-assortm...,"{""0"": ""https://www.auveco.com/site/item-images...","{""0"": {""Pcs/Unit"": ""1""}, ""1"": {""Includes"": ""19..."


In [14]:
crawler_status = 'ok'

crawler_status

'ok'

In [15]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [16]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_part.xlsx', index=False)

output_correct

,No,SKU,Title,OEM,Picture,Url,Json_Src,Json_Description
0,1,1043,Metric Fastener Assortment withoutut Compartme...,,,https://www.auveco.com/metric-fastener-assortm...,"{""0"": ""https://www.auveco.com/site/item-images...","{""0"": {""Pcs/Unit"": ""1""}, ""1"": {""Includes"": ""19..."


In [17]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_part_error.xlsx', index=False)

output_error

""
